In [44]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from keras import regularizers

(X_train_full, y_train_full), (X_test, y_test) = datasets.cifar10.load_data()

print(X_train_full.shape)

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

X_val, X_train = X_train_full[:20000] / 255.0, X_train_full[20000:] / 255.0
y_val, y_train = y_train_full[:20000], y_train_full[20000:]
X_test = X_test / 255.0

(50000, 32, 32, 3)


In [45]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, GlobalMaxPooling2D
from tensorflow.keras.regularizers import l2

def build_mobilenet(input_shape=(32, 32, 3), dropout_rate = 0.8, l2_reg = 0.001):
    base_model = MobileNetV2(weights=None, include_top=False, input_shape=input_shape)

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(256, activation='relu', kernel_regularizer=l2(l2_reg))(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)

    x = Dense(128, activation='relu', kernel_regularizer=l2(l2_reg))(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    
    output = Dense(10, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    return model

input_shape = (32, 32, 3)
dropout_rate = 0.5
l2_reg = 0.0001

model = build_mobilenet(input_shape=input_shape, dropout_rate= dropout_rate)
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 Conv1 (Conv2D)                 (None, 16, 16, 32)   864         ['input_8[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 16, 16, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 16, 16, 32)   0           ['bn_Conv1[0][0]']               
                                                                                            

In [46]:
tf.random.set_seed(1)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 25)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [47]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=64, callbacks=[callback])

Epoch 1/100
469/469 [==============================] - 29s 49ms/step - loss: 2.9905 - accuracy: 0.1856 - val_loss: 2.8033 - val_accuracy: 0.1014
Epoch 2/100
469/469 [==============================] - 22s 47ms/step - loss: 2.3193 - accuracy: 0.2981 - val_loss: 2.6303 - val_accuracy: 0.1006
Epoch 3/100
469/469 [==============================] - 23s 49ms/step - loss: 1.9695 - accuracy: 0.3660 - val_loss: 2.5186 - val_accuracy: 0.1006
Epoch 4/100
469/469 [==============================] - 22s 47ms/step - loss: 1.7601 - accuracy: 0.4137 - val_loss: 2.4524 - val_accuracy: 0.1021
Epoch 5/100
469/469 [==============================] - 20s 44ms/step - loss: 1.6199 - accuracy: 0.4518 - val_loss: 2.4301 - val_accuracy: 0.1021
Epoch 6/100
469/469 [==============================] - 21s 44ms/step - loss: 1.5067 - accuracy: 0.4875 - val_loss: 2.4577 - val_accuracy: 0.1021
Epoch 7/100
469/469 [==============================] - 19s 41ms/step - loss: 1.4360 - accuracy: 0.5148 - val_loss: 2.3801 - val_ac

Once tuning is finished, evaluate on testing set.

In [48]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 3s 10ms/step - loss: 1.7698 - accuracy: 0.6765


[1.7698378562927246, 0.6765000224113464]

In [49]:
model.save('MobileNet.h5')